In [50]:
%load_ext Cython
# import math
import random
from random import shuffle
import numpy as np

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [51]:
class Domino:
    
    def buy(self,hand, n, pieces):
        i = 0
        while i<n:
            hand.append(pieces[i])
            i=i+1
        del pieces[0:n]
        return hand

    #recebe estado do jogo, retorna ações possíveis
    def possibleActions(self,state,player):
        status = state[0]
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
        l_end = state[4]
        r_end = state[5]
        actions = []
        index = -1

        if(l_end==r_end==-1):
            for piece in hand:
                index +=1
                actions.append((piece[0],piece[1],index))
            return actions

        for piece in hand:
            index +=1
            #peça dupla
            if(piece[0]==piece[1]):
                if(piece[0]==l_end):
                    actions.append((piece[1],"left",index))
                elif(piece[1]==r_end):
                    actions.append((piece[0],"right",index))
                continue
            if(piece[0]==l_end):
                actions.append((piece[1],"left",index))
                if(l_end==r_end): #evitar duplicacao de mesmas ações na esq e dir
                    continue
            if(piece[1]==l_end):
                actions.append((piece[0],"left",index))
                if(l_end==r_end):
                    continue
            if(piece[0]==r_end):
                actions.append((piece[1],"right",index))
                if(l_end==r_end):
                    continue
            if(piece[1]==r_end):
                actions.append((piece[0],"right",index))
                if(l_end==r_end):
                    continue
        '''
        pos 2 e 3 no actions se referem a nova ponta na mesa:
        exemplo: campo 1-5 , mão : (1,2),(5,5)
        nesse caso a peça (1,2) se encaixa no lado esquerdo, sendo
        a nova ponta esquerda o 2. A representação será: (0,1), pois
        0 é a posição da tupla na mão, e o 1 representa a posição na tupla,
        que no caso é o direito

        '''
        if not actions:
            actions.append(None)
        return actions

    def termination(self,state):#se a partida ja terminou
        status = state[0]
        p1_hand = state[1]
        p2_hand = state[2]
        l_end = state[4]
        r_end = state[5]
        if (status >= 3):
            return True  
        return False
    
    def reward(self,state,player):#recompensa por estar nesse estado
        #em caso de vitoria
        status = state[0]
        p1_hand = state[1]
        p2_hand = state[2]
        l_end = state[4]
        r_end = state[5]
        if(status == 1):
            return 0
        
        if(status == 2):#avaliar block    
            blockedPlayer = state[6]
            if(blockedPlayer == "p1"):
                if(player == "p1"):#fui bloqueado
                    return -2
                else:
                    return 2
            elif(blockedPlayer == "p2"):
                if(player == "p2"):#fui bloqueado
                    return -2 
                else:
                    return 2
            return 0
            
        if(status >= 3):
            p1_hand = state[1]
            p2_hand = state[2]
            lighestP1Value = 13
            lighestP2Value = 13
            lighestP1Tile = 13
            lighestP2Tile = 13
            p1_total_value = 0
            p2_total_value = 0
            for piece in p1_hand:
                value = piece[0]+piece[1]
                p1_total_value+=value
            for piece in p2_hand:
                value = piece[0]+piece[1]
                p2_total_value+=value
            if(p1_total_value<p2_total_value):
                state[0] = 4#player 1 venceu tendo a menor mao
                if(player == "p1"):
                    return 3 + p2_total_value
                else:
                    return -3 - p2_total_value # menos os pontos q p2 deu pro oponente
            elif(p1_total_value>p2_total_value):
                state[0] = 5#player 2 venceu tendo a menor mao
                if(player == "p1"):
                    return -3 - p1_total_value 
                else:
                    return 3 + p1_total_value
            else:#empate
                for piece in p1_hand:
                    if(piece[0]<lighestP1Value):
                        lighestP1Value = piece[0]
                        lighestP1Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP1Value):
                        lighestP1Value = piece[1]
                        lighestP1Tile = piece[0]+piece[1]
                        
                for piece in p2_hand:
                    if(piece[0]<lighestP2Value):
                        lighestP2Value = piece[0]
                        lighestP2Tile = piece[0]+piece[1]
                    if(piece[1]<lighestP2Value):
                        lighestP2Value = piece[1]
                        lighestP2Tile = piece[0]+piece[1]
                        
                if(lighestP1Value<lighestP2Value):
                    state[0] = 4#player 1 venceu tendo o menor valor entre o par das pecas    
                    if(player == "p1"):
                        return 3 + p2_total_value
                    else:
                        return -3 - p2_total_value
                elif(lighestP1Value>lighestP2Value):
                    state[0] = 5#player 2 venceu tendo o menor valor entre o par das pecas
                    if(player == "p1"):
                        return -3 - p1_total_value
                    else:
                        return 3 + p1_total_value
                else:
                    if(lighestP1Tile<lighestP2Tile):#menor valor da peça como um todo
                        state[0] = 4
                        if(player == "p1"):
                            return 3 + p2_total_value
                        else:
                            return -3 - p2_total_value
                    else:#nao pode haver empate
                        state[0] = 5
                        if(player == "p1"):
                            return -3 - p1_total_value
                        else:
                            return 3 + p1_total_value

    def startGame(self):
        status = 1 #1=in progress; 2=one player blocked;3=two players blocked;4/5=p1 won/p2 won
        pieces = [(x,y) for x in range(7) for y in range(x,7)]
        random.shuffle(pieces)
        p1_hand = self.buy([],7,pieces)
        p2_hand = self.buy([],7,pieces)
        field = []
        l_end = -1
        r_end = -1
        blockedPlayer = "None" #jogador que foi bloqueado
        state = [status, p1_hand, p2_hand, field, l_end, r_end,blockedPlayer]
        return state

    def playGame(self,state,action,player):
        status = state[0]
        if(status >= 3): #jogo acabou
            return state
        if(player =="p1"):
            hand = state[1]
        else:
            hand = state[2]
            
        field = state[3]
        l_end = state[4]
        r_end = state[5]
        
        if(action[0] is None):#foi bloqueado
            if(status == 2):
                state[0] = 3
            elif(status == 1):
                state[0] = 2
                state[6] = player  
                
            return state
        
        if(status == 2): # o oponente/ou eu foi bloqueado, mas um de nos tem peça
            state[0] = 1
        p_index = action[2]  
        if(l_end==r_end==-1):
            orientation = "left"
        else:
            orientation = action[1]
        p = hand[p_index]
        field.append(p)
        hand.remove(p)
        if (l_end==-1 and r_end==-1):
            l_end, r_end = p
        elif (orientation == "right"):#ori e o lado desejado a manter na ponta
            r_end=action[0]
        elif (orientation == "left"):
            l_end=action[0]
                
        if(player =="p1"):
            state[1] = hand
        else:
            state[2] = hand
            
        if(len(hand) ==0): #zerou a mao, acabou o jogo
            state[0] = 3
        state[3] = field
        state[4] = l_end
        state[5] = r_end
        return state



In [52]:
%%cython
cimport numpy

cpdef cysum(numpy.ndarray[double] A,numpy.ndarray[double] B):
    """Compute the sum of an array"""
    cdef int count=0
    smax = B.shape[0]
    
    for i in range(A.shape[0]):
        for j in range(i,A.shape[0]):
            if(A[i]==1 and A[j]==1):
                B[count] = 1
            else:
                B[count] = 0    
            count = count + 1
    return B

In [143]:
# class Features:

def __init__(self,beta,featureWeights,combinationWeigths):
    self.data = []
    self.beta = beta
    self.featureWeights = featureWeights
    self.combinationWeigths = combinationWeigths
    self.numFeatures = featureWeights.size
    self.numFeaturesComb = combinationWeigths.size
    #self.posi =  np.zeros(630)
    #self.posj =  np.zeros(630)

def getFeaturesWeights(self):
    return self.featureWeights

def setFeaturesWeights(self,featureWeights):
    self.featureWeights = featureWeights

def getCombWeights(self):
    return self.combinationWeigths

def setCombWeights(self,combinationWeigths):
    self.combinationWeigths = combinationWeigths

def generateFeatures(self,state,action,player):
    atomFeatures = self.atomicFeatures(state,action,player)
    pairwiseFeatures = self.pairAndFeatures(atomFeatures)
    return pairwiseFeatures
    #result = generateAndFeatures(state,action,player)


def atomicFeatures(self,state,action,player):
    result =  np.zeros(self.numFeatures)
    if(action[0]==None):
        return result
    l_end = state[4]
    r_end = state[5]
    field = state[3]
    blockedPlayer = state[6]
    status = state[0]

    if(player == "p1"):
        hand = state[1]
        handOp = state[2]
    elif(player == "p2"):
        hand = state[2]
        handOp = state[1]

    pos = action[2]
    bone =hand.pop(pos)
    opActs = self.realCount(handOp,bone,l_end,r_end)
    myActs = self.realCount(hand,bone,l_end,r_end)
    if(l_end==r_end==-1):
        l_end = action[0]
        r_end = action[1]
    elif(action[1]=="left"):
        #substitui uma das pontas pela nova ação
        l_end = action[0]
    else:
        r_end = action[0]


    #Mão

    result[0] = self.NumDouble(hand,bone,lambda x: x==0)
    result[1] = self.NumDouble(hand,action,lambda x: x==1)
    result[2] = self.NumPoints(hand,bone,lambda x: x>20,lambda x: x<20)
    result[3] = self.NumPoints(hand,bone,lambda x: x>10,lambda x: x<10)
    result[4] = self.NumPoints(hand,bone,lambda x: x>5 ,lambda x: x<5)
    j = 6
    for i in range(5,12):
        result[i] = self.NumActions(hand,bone,l_end,r_end,MyActs,j)
        j-=1
    #Campo

    result[12] = l_end == r_end #msm se ja estivesse antes, sua ação continuou
    result[13] = self.NumPlayed(field,bone[0],3)#saber q 2 ou mais ja foram jogadas pode ajudar
    result[14] = self.NumPlayed(field,bone[0],4)#a dar coragem
    result[15] = self.NumPlayed(field,bone[0],5)
    if(bone[0]!=bone[1]):
        result[16] = self.NumPlayed(field,bone[1],3)
        result[17] = self.NumPlayed(field,bone[1],4)
        result[18] = self.NumPlayed(field,bone[1],5)
    else:
        result[16] = False # é uma mentira, mas é pra n influenciar a favor de peças duplas
        result[17] = False
        result[18] = False

    #Mao do oponente

    result[19] = self.canBeBlocked(hand,handOp,action)#cuidado na 1 rodada como p1
    result[20] = self.blockOne(handOp,action)


    result[21] = self.blocks(handOp,l_end,r_end)
    j = -6
    for i in range(22,30):
        result[i] = self.NumActions(handOp,bone,l_end,r_end,opActs,j)
        j+=1



    if(action[0]!=None):
        hand.insert(pos,bone)

    return result


#Geracao de caracteristicas por combinacao de pares And
def pairAndFeatures(self,atomFeatures):
    pairwiseFeatures =  np.zeros(self.numFeaturesComb)
    count = 0
    pairwiseFeatures = cysum(atomFeatures,pairwiseFeatures)
    return pairwiseFeatures

def realCount(self,hand,bone,l_end,r_end):
    count = 0
    for piece in hand:
        if(self.isPlayable(bone,l_end,r_end)):
            count +=1
    return count

def NumDouble(self,hand,bone,exp):
    if(bone[0]!=bone[1]):#se a peça retirada foi msm uma dupla
        return False
    count = 0
    for piece in hand:
        if(piece[0] == piece[1]):
            count +=1
    if(exp(count)):
        return True
    return False

def NumPoints(self,hand,bone,exp1,exp2):
    bonePoints = bone[0]+bone[1]
    points = 0
    for piece in hand:
        points += piece[0] + piece[1]
    if(exp1(points+bonePoints)):#se antes era maior
        if(exp2(points+bonePoints)):#e agora e menor
            return True #entao foi essa ação a causadora
    return False

def NumPlayed(self,field,value,num):
    count = 0
    for piece in field:
        if(piece[0]==value or piece[1]==value):
            count+=1
    if(count == num):
        return True
    else:
        return False


def hasDouble(self,number,hand):
    for piece in hand:
        if((piece[0] == piece[1]) and (piece[0] == number)):
            return True
    return False

def hasPieceValue(self,hand,exp):
    for piece in hand:
        value = piece[0] + piece[1]
        if(exp(value)):
            return True
    return False

def hasHandValue(self,hand,exp):
    value = 0
    for piece in hand:
        value += piece[0] + piece[1]
    if(exp(value)):
        return True
    else:
        return False

def blockOne(self,handOp,action):
    for piece in handOp:
        if(piece[0]==action[0] or piece[1]==action[0]):
            return False
    return True

def blocks(self,hand,l_end,r_end):
    for piece in hand:
        if(self.isPlayable(piece,l_end,r_end)):
            #se encontra alguma peça valida na mao
            return False # então ainda não estou bloqueado
    return True #nao encontrou


def NumActions(self,handOp,bone,l_end,r_end,numAct,num):
    if(numAct == 0):
        return False
    newCount = 0
    for piece in handOp:
        if(self.isPlayable(bone,l_end,r_end)):
            newCount +=1    

    change = newCount - numAct
    if(change == num):
        return True
    else:
        return False

    return False #se tem exatamente aquele numero

def canBeBlocked(self,hand,handOp,action):#se o op for bloqueado não importa
    sideValue = 0
    cond = False
    if(action[1]=="left"):#trata somente o lado em que a peça for colocada
        sideValue = action[0]
    else:
        sideValue = action[0]
    for pieceOp in handOp:#buscar na mao do op se ele tem uma peça que pode me bloquear
        if(pieceOp[0]==sideValue):
            sideValue = pieceOp[1]#substitui o canto pela peça do oponente
            for piece in hand:
                if(piece[0]==sideValue or piece[1]==sideValue):
                    cond = True#axou uma peça na mao jogavel
                    break
            if(not cond):#eu n tenho uma peça jogavel contra essa do oponente
                return True
        elif(pieceOp[1]==sideValue):
            sideValue = pieceOp[0]
            for piece in hand:
                if(piece[0]==sideValue or piece[1]==sideValue):
                    cond = True
                    break
            if(not cond):#eu n tenho uma peça jogavel contra essa do oponente
                return True

    return False #axou uma peça na mao jogavel

def HandSize(self,hand,exp):
    lenght = len(hand)
    return exp(lenght)

def shortPiece(self,number,handOp):
    for piece in handOp:
        if(piece[0]==number or piece[1]==number):
            return False
    return True #op n tem nenhuma peça dese tipo


def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
    if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
        return True
    return False   

In [4]:
class Features2:
    
    def __init__(self,beta,featureWeights,combinationWeigths):
        self.data = []
        self.beta = beta
        self.featureWeights = featureWeights
        self.combinationWeigths = combinationWeigths
        self.numFeatures = featureWeights.size
        self.numFeaturesComb = combinationWeigths.size
        #self.posi =  np.zeros(630)
        #self.posj =  np.zeros(630)
        
    def getFeaturesWeights(self):
        return self.featureWeights
    
    def setFeaturesWeights(self,featureWeights):
        self.featureWeights = featureWeights
        
    def getCombWeights(self):
        return self.combinationWeigths
    
    def setCombWeights(self,combinationWeigths):
        self.combinationWeigths = combinationWeigths
    
    def generateFeatures(self,state,action,player):
        atomFeatures = self.atomicFeatures(state,action,player)
        pairwiseFeatures = self.pairAndFeatures(atomFeatures)
        return pairwiseFeatures
        #result = generateAndFeatures(state,action,player)
        
    
    def atomicFeatures(self,state,action,player):
        result =  np.zeros(self.numFeatures)
        l_end = state[4]
        r_end = state[5]
        if(player == "p1"):
            hand = state[1]
            handOp = state[2]
        elif(player == "p2"):
            hand = state[2]
            handOp = state[1]
        if(action[0]!=None):#quando nao for bloqueado    
            pos = action[2]
            if(l_end==r_end==-1):
                pos = action[2]
                l_end = action[0]
                r_end = action[1]
            elif(action[1]=="left"):
            #substitui uma das pontas pela nova ação
                l_end = action[0]
            else:
                r_end = action[0]
            #print "pos: "+str(pos)
            value =hand.pop(pos)
        """
        Características a partir de informações imperfeitas.
            Somente do que é visível a um jogador comum:
                Campo, peças de suas mãos, quantidade de peças do oponente
        """
        #valida caracteristicas de acordo com o estado submetido e as retorna
        
        result[0] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==1)
        result[1] = self.NumDouble(hand,action,l_end,r_end,lambda x: x==2)
        result[2] = self.NumDouble(hand,action,l_end,r_end,lambda x: x>=3)
        #impossibilita qualquer peça de ser jogada
        result[3] = self.blocks(hand,action,l_end,r_end)
        
        result[4] = self.NumActions(hand,l_end,r_end,lambda x: x==1)
        result[5] = self.NumActions(hand,l_end,r_end,lambda x: x==2)
        result[6] = self.NumActions(hand,l_end,r_end,lambda x: x>=3)
       
        for i in range(7,14):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.hasDouble(i-7,hand)
                
        """
        Características a partir de informações perfeitas.
        Sabe-se da mão do oponente, de toda e qualquer informação necessária
        para se tomar uma decisão
        """
        result[14] = self.blocks(handOp,action,l_end,r_end)
        result[15] = self.NumActions(handOp,l_end,r_end,lambda x: x==1)
        result[16] = self.NumActions(handOp,l_end,r_end,lambda x: x==2)
        result[17] = self.NumActions(handOp,l_end,r_end,lambda x: x>=3)
        for i in range(18,25):
            #quais peças duplas temos na mão, de 0:0 a 6:6
            result[i] = self.shortPiece(i-18,handOp)
        
        
        result[25] = self.hasPieceValue(hand,lambda x: x>=9)
        result[26] = self.hasPieceValue(hand,lambda x: x<=3)
        result[27] = self.hasHandValue(hand,lambda x: x<=4)
        result[28] = self.hasHandValue(hand,lambda x: x>=12 and x<=19)
        result[29] = self.hasHandValue(hand,lambda x: x>=20)
            
        result[30] = self.hasPieceValue(handOp,lambda x: x>=9)
        result[31] = self.hasPieceValue(handOp,lambda x: x<=3)
        result[32] = self.hasHandValue(handOp,lambda x: x<=4)
        result[33] = self.hasHandValue(handOp,lambda x: x>=12 and x<=19)
        result[34] = self.hasHandValue(handOp,lambda x: x>=20)
        
        #result[35] = self.HandSize(hand,lambda x: x<=3)
        #result[36] = self.HandSize(hand,lambda x: x>3)
        #result[37] = self.HandSize(handOp,lambda x: x<=3)
        #result[38] = self.HandSize(handOp,lambda x: x>3)
        
        
        if(action[0]!=None):
            hand.insert(pos,value)
        
        return result
    
    
    #Geracao de caracteristicas por combinacao de pares And
    def pairAndFeatures(self,atomFeatures):
        pairwiseFeatures =  np.zeros(self.numFeaturesComb)
        count = 0
        pairwiseFeatures = cysum(atomFeatures,pairwiseFeatures)
        return pairwiseFeatures
    
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def HandSize(self,hand,exp):
        lenght = len(hand)
        return exp(lenght)
    
    def hasDouble(self,number,hand):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def hasPieceValue(self,hand,exp):
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                return True
        return False
    
    def hasHandValue(self,hand,exp):
        value = 0
        for piece in hand:
            value += piece[0] + piece[1]
        if(exp(value)):
            return True
        else:
            return False
    
    def blocks(self,hand,action,l_end,r_end):
        if(action[0] is None):
            return True
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
  
        return True #nao encontrou
        
    def NumActions(self,hand,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
        if(exp(count)):#se tem mais que o numero de peças designado
            return True
        return False #se tem exatamente aquele numero
    
    def shortPiece(self,number,handOp):
        for piece in handOp:
            if(piece[0]==number or piece[1]==number):
                return False
        return True #op n tem nenhuma peça dese tipo
        
          
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
            return True
        return False
    
    def generateAndFeatures(self,state,action,player):
        result1 = self.generateAtomic(state,action,player)
        f = self.featureCombWeights
        #for i in range(0,f.size):
         #   if(result1[x[i]] == 1 and result1[y[i]] == 1 ):
                #f[i] = 1
            #else:
                #f[i] = 0
       

In [5]:
class Features3:
    
    def __init__(self,beta,featureWeights,combinationWeigths):
        self.data = []
        self.beta = beta
        self.featureWeights = featureWeights
        self.combinationWeigths = combinationWeigths
        self.numFeatures = featureWeights.size
        self.numFeaturesComb = combinationWeigths.size
        #self.posi =  np.zeros(630)
        #self.posj =  np.zeros(630)
        
    def getFeaturesWeights(self):
        return self.featureWeights
    
    def setFeaturesWeights(self,featureWeights):
        self.featureWeights = featureWeights
        
    def getCombWeights(self):
        return self.combinationWeigths
    
    def setCombWeights(self,combinationWeigths):
        self.combinationWeigths = combinationWeigths
    
    def generateFeatures(self,state,action,player):
        atomFeatures = self.atomicFeatures(state,action,player)
        pairwiseFeatures = self.pairAndFeatures(atomFeatures)
        return pairwiseFeatures
        #result = generateAndFeatures(state,action,player)
        
    
    def atomicFeatures(self,state,action,player):
        result =  np.zeros(self.numFeatures)
        l_end = state[4]
        r_end = state[5]
        status = state[0]
        blockedPlayer = state[6]
        if(player == "p1"):
            hand = state[1]
            handOp = state[2]
        elif(player == "p2"):
            hand = state[2]
            handOp = state[1]
        if(action[0]!=None):#quando nao for bloqueado    
            pos = action[2]
            if(l_end==r_end==-1):
                pos = action[2]
                l_end = action[0]
                r_end = action[1]
            elif(action[1]=="left"):
            #substitui uma das pontas pela nova ação
                l_end = action[0]
            else:
                r_end = action[0]
            #print "pos: "+str(pos)
            value =hand.pop(pos)
        
        result[0] = self.blocks(hand,l_end,r_end)
        result[1] = self.NumActions(hand,l_end,r_end,lambda x: x==1)
        result[2] = self.NumActions(hand,l_end,r_end,lambda x: x==2)
        result[3] = self.NumActions(hand,l_end,r_end,lambda x: x>=3)
        result[4] = self.blocks(handOp,l_end,r_end)
        result[5] = self.NumActions(handOp,l_end,r_end,lambda x: x==1)
        result[6] = self.NumActions(handOp,l_end,r_end,lambda x: x==2)
        result[7] = self.NumActions(handOp,l_end,r_end,lambda x: x>=3)
        for i in range(8,15):
            result[i] = self.shortPiece(i-18,handOp)
    
        result[15] = self.hasHandValue(hand,lambda x: x<=4)
        result[16] = self.hasHandValue(hand,lambda x: x>=20)
        result[17] = self.hasHandValue(handOp,lambda x: x<=4)
        result[18] = self.hasHandValue(handOp,lambda x: x>=20)
        #result[19] = self.canBeBlocked(hand,handOp,l_end,r_end)
        
        #result[19] = self.HandSize(hand,lambda x: x<=3)
        #result[20] = self.HandSize(hand,lambda x: x>3)
        #result[21] = self.HandSize(handOp,lambda x: x<=3)
        #result[22] = self.HandSize(handOp,lambda x: x>3)
        #if(status==2):# so valido com o op em condição de block
            #result[23] = self.blockMyself(hand,handOp,blockedPlayer,player,l_end,r_end)
        #else:
            #result[23] = False
        
        if(action[0]!=None):
            hand.insert(pos,value)
        
        return result
    
    
    #Geracao de caracteristicas por combinacao de pares And
    def pairAndFeatures(self,atomFeatures):
        pairwiseFeatures =  np.zeros(self.numFeaturesComb)
        count = 0
        pairwiseFeatures = cysum(atomFeatures,pairwiseFeatures)
        return pairwiseFeatures
    
    
    def NumDouble(self,hand,action,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(piece[0] == piece[1]):
                count +=1
        if(exp(count)):
            return True
        return False
    
    def canBeBlocked(self,hand,handOp,l_end,r_end):#tomar cuidado, caracteristica pesada
        for pieceOp in handOp:
            if(pieceOp[0]==l_end):#substitui um dos cantos pela peça do oponente
                l_end = pieceOp[1]#lado oposto ao que encaixa
            elif(pieceOp[0]==r_end):
                r_end = pieceOp[1]
            if(self.blocks(hand,l_end,r_end)):#se n puder fazer jogada
                return True
            if(pieceOp[1]==l_end):#substitui um dos cantos pela peça do oponente
                l_end = pieceOp[0]
            elif(pieceOp[1]==r_end):
                r_end = pieceOp[0]
            if(self.blocks(hand,l_end,r_end)):#se n puder fazer jogada
                return True
        return False
    
    def blockMyself(self,hand,handOp,blockedPlayer,player,l_end,r_end):
        if(player !=blockedPlayer):
            if(len(hand)>1):#se ainda tem alguma peça na mao
                if(self.blocks(hand,l_end,r_end) and self.blocks(handOp,l_end,r_end)):#acabar com o jogo
                    return True
        return False
    
    def HandSize(self,hand,exp):
        lenght = len(hand)
        return exp(lenght)
    
    def hasDouble(self,number,hand):
        for piece in hand:
            if((piece[0] == piece[1]) and (piece[0] == number)):
                return True
        return False
    
    def hasPieceValue(self,hand,exp):
        for piece in hand:
            value = piece[0] + piece[1]
            if(exp(value)):
                return True
        return False
    
    def hasHandValue(self,hand,exp):
        value = 0
        for piece in hand:
            value += piece[0] + piece[1]
        if(exp(value)):
            return True
        else:
            return False
    
    def blocks(self,hand,l_end,r_end):
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                #se encontra alguma peça valida na mao
                return False # então ainda não estou bloqueado
  
        return True #nao encontrou
        
    def NumActions(self,hand,l_end,r_end,exp):
        count = 0
        for piece in hand:
            if(self.isPlayable(piece,l_end,r_end)):
                count +=1
        if(exp(count)):#se tem mais que o numero de peças designado
            return True
        return False #se tem exatamente aquele numero
    
    def shortPiece(self,number,handOp):
        for piece in handOp:
            if(piece[0]==number or piece[1]==number):
                return False
        return True #op n tem nenhuma peça dese tipo
        
          
    def isPlayable(self,piece,l_end,r_end):#se dado a peça e as duas pontas, ela é jogavel
        if(piece[0]==l_end or piece[0]==r_end or piece[1]==l_end or piece[1]==r_end):
            return True
        return False
    
    def generateAndFeatures(self,state,action,player):
        result1 = self.generateAtomic(state,action,player)
        f = self.featureCombWeights
        #for i in range(0,f.size):
         #   if(result1[x[i]] == 1 and result1[y[i]] == 1 ):
                #f[i] = 1
            #else:
                #f[i] = 0
       

In [144]:
class Control:
    def __init__(self,player,features,featuresType,gamma,tdlambda):
        self.player = player
        self.features = features
        self.featuresVector = np.zeros(1)
        self.featuresType = featuresType
        
        self.gamma = gamma
        self.tdlambda = tdlambda
        if(featuresType == "atomic"):
            self.eTraces = np.zeros(features.getFeaturesWeights().size)
            self.auxFeaturesVector =  np.zeros(features.getFeaturesWeights().size)
        else:
            self.eTraces = np.zeros(features.getCombWeights().size)
            self.auxFeaturesVector =  np.zeros(features.getCombWeights().size)
    #dado o jogador, sua acao, o estado e suas caracteristicas, retorne o valor dessa acao
    #e necessario diferencia os jogadores pois ele precisa examinar 2 hands 
    #o act n especifica de que hand e
    def getFeatures(self):
        return self.features
    
    def getPlayer(self):
        return self.player
    
    def getTraces(self):
        return self.eTraces
    
    def resetTraces(self):
        self.eTraces = np.zeros(features.size)
        
    def fillTraces(self):
        self.eTraces[self.featuresVector>0] +=1
        
    def generateVector(self,state,act):
        if(self.featuresType == "atomic"):
            self.featuresVector = self.features.atomicFeatures(state,act,self.player)
            print self.featuresVector.size
            
        elif(self.featuresType == "combination"):
            self.featuresVector = self.features.generateFeatures(state,act,self.player)
                
    def qValue(self,state,act):
        if(self.featuresType == "atomic"):
            featureVals = self.features.atomicFeatures(state,act,self.player)
            val = np.dot (self.features.getFeaturesWeights(), featureVals)    # value before action
        elif(self.featuresType == "combination"):
            featureVals = self.features.generateFeatures(state,act,self.player)
            val = np.dot (self.features.getCombWeights(), featureVals)
        return val

    def eGreedyPicker(self,actions,state):
        rand = np.random.random_sample()
        bestValue = 0 #tomar cuidado aqui
        cond = False
        if(rand > e):
            for act in actions:  
                value = self.qValue(state,act)
                if(not cond):
                    bestValue = value
                    choosenAct = act
                    cond = True
                if(value> bestValue):
                    bestValue = value
                    choosenAct = act
        else:
            choosenAct = random.choice(actions)

        return choosenAct


    def policyAct(self,state):
        actions = dominoes.possibleActions(state,self.player)
        #escolhe a partir dos valores na q-value com tecnica e-greedy
        if(actions[0] is None):
            return actions
        else:
            choosenAct = self.eGreedyPicker(actions,state)

        return choosenAct

    def step(self,state,act):
        state = dominoes.playGame(state,act,self.player)                  # do selected action
        return state

    def alpha(self,vector):#calcua taxa de aprendizado de acordo com quantidade de features atuais
        print "vector: "+str(vector)
        unique, counts = np.unique(vector, return_counts=True)
        if((counts[0])==self.featuresVector.size):
            return 0.01
        a = 0.1/(counts[1]) #constante por numero total de 1
        return a
    
    def decayTraces(self):
        if(self.tdlambda>0):
            feat = self.featuresVector
            self.eTraces[feat>0]=self.eTraces[feat>0]*self.gamma*self.tdlambda
            self.auxFeaturesVector = self.eTraces
            self.auxFeaturesVector[self.featuresVector==0] = 0#retorna trace das carac atuais
            return self.auxFeaturesVector 
        else:
            return self.featuresVector
        
    #A recompensa é dada após a ação dos 2 players
    #não é recompensa pela ação de 1 player, mas dos dois
    def evaluate(self,state,val,act):
        self.fillTraces()
        r = dominoes.reward(state,self.player)
        next_action = self.policyAct(state)                 
        new_value = self.qValue(state,next_action)
        target = r+ self.gamma * new_value        # gamma = 0.9
        delta = target - val                            # prediction error
        if(self.featuresType == "atomic"):
            featureWeights = self.features.getFeaturesWeights()
            featureWeights += self.alpha(self.featuresVector) * delta *self.decayTraces()
            self.features.setFeaturesWeights(featureWeights)
        elif(self.featuresType == "combination"):
            featureWeights = self.features.getCombWeights()
            featureWeights += self.alpha(self.featuresVector) * delta *self.decayTraces()
            self.features.setCombWeights(featureWeights)
        elements = [state,new_value,next_action,r]
        return elements



In [147]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
featureWeights1 = np.zeros(30)
featureWeights2 = np.zeros(30)
featureComb= np.zeros(190)
features1 = Features(1,featureWeights1,featureComb)
features2 = Features(1,featureWeights2,featureComb)
controlP1 = Control("p1",features1,"atomic",0.9,0)
controlP2 = Control("p2",features2,"atomic",0.9,0)
r = 0
duration = 0
#self-play
firstplay = 0
accumulatedRewardP1 = 0
accumulatedRewardP2 = 0
P1Wins = 0
P2Wins = 0
for i in range(0,1):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)#2 jogadores compartilhando a msm rede neural,mas so 1 avaliando
    
    actP1 = controlP1.policyAct(state)
    
    val1 = controlP1.qValue(state,actP1)
    flag = 0
    for j in range(0,1):
    #while(dominoes.termination(state) == False):
        print actP1
        print state
        controlP1.generateVector(state,actP1)    
        state = controlP1.step(state,actP1)
        
        if(flag==1):
            state,val2,actP2,r =controlP2.evaluate(state,val2,actP2)
            controlP2.generateVector(state,actP2)
            accumulatedRewardP2+=r
        if(flag==0):
            actP2 = controlP2.policyAct(state)
            controlP2.generateVector(state,actP2)
            flag = 1
        val2 = controlP2.qValue(state,actP2)
        state = controlP2.step(state,actP2)
        state,val1,actP1,r =controlP1.evaluate(state,val1,actP1)
        accumulatedRewardP1 +=r
        
        if(dominoes.termination(state) == True):
            state,val2,actP2,r =controlP2.evaluate(state,val2,actP2)
            accumulatedRewardP2+=r
            if(state[0]==4):
                P1Wins+=1
            else:
                P2Wins+=1
        
featureWeights1 = controlP1.getFeatures().getFeaturesWeights()
featureWeights2 = controlP2.getFeatures().getFeaturesWeights()
print controlP1.getTraces()
print state
print "Resulting weight1: "+str(featureWeights1)
print "Resulting weight2: "+str(featureWeights2)
print "P1victories: "+str(P1Wins) + " P1 Accumulated reward: "+ str(accumulatedRewardP1)
print "P2victories: "+str(P2Wins) + " P2 Accumulated reward: "+ str(accumulatedRewardP2)
if(P1Wins>P2Wins):
    winnerWeigth = featureWeights1
else:
    winnerWeigth = featureWeights2
np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\actionAgent', winnerWeigth)

(0, 5, 0)
[1, [(0, 5), (5, 5), (2, 4), (0, 0), (1, 5), (4, 6), (1, 2)], [(4, 4), (5, 6), (1, 4), (1, 6), (0, 3), (0, 2), (6, 6)], [], -1, -1, 'None']
30
30
vector: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[1, [(5, 5), (2, 4), (0, 0), (1, 5), (4, 6), (1, 2)], [(4, 4), (1, 4), (1, 6), (0, 3), (0, 2), (6, 6)], [(0, 5), (5, 6)], 0, 6, 'None']
Resulting weight1: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Resulting weight2: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
P1victories: 0 P1 Accumulated reward: 0
P2victories: 0 P2 Accumulated reward: 0


In [ ]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
featureWeights = np.zeros(35)
featureComb= np.zeros(630)
CombWeightsP1 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNetPairwise35.npy')
CombWeightsP2 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNetPairwise35.npy')
features1 = Features(1,featureWeights,CombWeightsP1)
features2 = Features(1,featureWeights,CombWeightsP2)
controlP1 = Control("p1",features1,"combination")
controlP2 = Control("p2",features2,"combination")
r = 0
duration = 0
#self-play
firstplay = 0
for i in range(0,10000):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)#2 jogadores compartilhando a msm rede neural,mas so 1 avaliando

    actP1 = controlP1.policyAct(state)  
    val1 = controlP1.qValue(state,actP1)
    flag = 0
    while(dominoes.termination(state) == False):
        if(firstplay==0):#salva vetor de carac antes de mudar
            controlP1.generateVector(state,actP1)
        else:
            controlP2.generateVector(state,actP2)
        state = controlP1.step(state,actP1)        
        if(flag==0):
            actP2 = controlP2.policyAct(state)
            flag = 1
        val2 = controlP2.qValue(state,actP2)
        state = controlP2.step(state,actP2)
        if(firstplay==0):
            state,val1,actP1 =controlP1.evaluate(state,val1,actP1)
            actP2 = controlP2.policyAct(state)
        else:
            state,val2,actP2 =controlP2.evaluate(state,val2,actP2)
            actP1 = controlP1.policyAct(state)
   
featureWeights = controlP1.getFeatures().getCombWeights()
print "Resulting weights: "+str(featureWeights)
np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNetPairwise35', featureWeights)

In [45]:
dominoes = Domino()
e = 0.0001  # aqui n ha aprendizagem
featureWeightsP1 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNetP1.npy')
featureWeightsP2 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\coadaptive3.npy')
featureComb= np.zeros(1)
featureWeights = np.zeros(19)
featuresP1 = Features2(1,featureWeightsP1,featureComb)
featuresP2 = Features3(1,featureWeights,featureWeightsP2)
controlP1 = Control("p1",featuresP1,"atomic",0,0)
controlP2 = Control("p2",featuresP2,"combination",0,0)
r = 0
duration = 0

P1Wins = 0
#firstplay =1
accumulatedRewardP1 = 0
accumulatedRewardP2 = 0
for i in range(0,1000):
    state = dominoes.startGame()
    firstplay = random.randint(0,1)
    while(dominoes.termination(state) == False):
        if(firstplay==0):
            actP1 = controlP1.policyAct(state)
            state = controlP1.step(state,actP1) 
            actP2 = controlP2.policyAct(state)
            state = controlP2.step(state,actP2)
        else:
            actP2 = controlP2.policyAct(state)
            state = controlP2.step(state,actP2)
            actP1 = controlP1.policyAct(state)
            state = controlP1.step(state,actP1)
        accumulatedRewardP1 += dominoes.reward(state,"p1")
        accumulatedRewardP2 +=dominoes.reward(state,"p2")
    if(state[0]==4):
        P1Wins+=1


print "weights: "+str(featureWeightsP1)
#print "weights: "+str(featureCombP2)
#np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet3', featureWeightsP1)
print "Player1 winnings: "+str(P1Wins)
print "Player1 rewards: "+str(accumulatedRewardP1)+" Player2 rewards: "+str(accumulatedRewardP2)

weights: [-0.09192529 -0.20659979 -0.18093689  1.51690381  1.39831485  1.3739857
  1.47406724  0.39218379  0.23245208 -0.12492196 -0.3718462  -0.41579741
 -0.3056125  -0.45784327 -0.13942727 -0.40643513 -0.63778834 -0.7466368
  0.22032436  0.21868294  0.06141763  0.12064992 -0.04507749 -0.01658206
  0.0762826  -0.27093872 -0.10939788  1.18334184 -0.36133493 -0.82256886
  0.30248491 -0.32532836 -0.90514442  0.6470053   1.17833832]
Player1 winnings: 666
Player1 rewards: 7834 Player2 rewards: -7834


In [ ]:
dominoes = Domino()
e = 0.0001  # aqui n ha aprendizagem
featureWeightsP1 = np.load('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNetP1.npy')
featureComb= np.zeros(1)
featureWeights = np.zeros(35)
featuresP1 = Features2(1,featureWeightsP1,featureComb)
controlP1 = Control("p1",featuresP1,"atomic")
controlP2 = Control("p2",featuresP1,"atomic")
r = 0
duration = 0

P1Wins = 0
firstplay = 0
for i in range(0,1):
    state = dominoes.startGame()
    #firstplay = random.randint(0,1)
    while(dominoes.termination(state) == False):
        if(firstplay==0):
            actP1 = controlP1.policyAct(state)
            state = controlP1.step(state,actP1) 
            print "State after p1 play: "+str(state)
            #print "State after p1 play: "+str(state[4]) + ","+str(state[5])
            #print "my hand: "+str(state[2])
            actionss2 = dominoes.possibleActions(state,"p2")
            if(actionss2[0] is None):
                actP2 = controlP2.policyAct(state)
                state = controlP2.step(state,actP2)
            else:
                actionPos=input("Choose Action: "+str(actionss2))
                state = controlP2.step(state,actionss2[actionPos])
        else:
            actP2 = controlP2.policyAct(state)
            state = controlP2.step(state,actP2)
            actP1 = controlP1.policyAct(state)
            state = controlP1.step(state,actP1)
        r += dominoes.reward(state,"p1")
        print "State after both plays: "+str(state)
    if(state[0]==4):
        P1Wins+=1


print "Resulting weights: "+str(featureWeightsP1)
print "Cumulative reward: "+str(r)
#np.save('C:\\Users\\LENOVO\\ReinforcementLearning\\neuralNet3', featureWeightsP1)
print "Player1 winnings: "+str(P1Wins)

In [ ]:
from timeit import timeit
from timeit import default_timer as timer
import numpy as np
import itertools
from itertools import combinations, chain
from scipy.misc import comb
def combs(a, r):
    """
    Return successive r-length combinations of elements in the array a.
    Should produce the same output as array(list(combinations(a, r))), but 
    faster.
    """
    a = np.asarray(a)
    dt = np.dtype([('', a.dtype)]*r)
    b = np.fromiter(itertools.combinations(a, r), dt)
    return b.view(a.dtype).reshape(-1, r)

def cartesian(arrays, out=None):
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

def comb_index(n, k):
    count = comb(n, k, exact=True)
    index = np.fromiter(chain.from_iterable(combinations(range(n), k)), 
                        int, count=count*k)
    return index.reshape(-1, k)

idx = comb_index(630, 2)
atomFeatures = np.zeros(35)
combfeatures = np.zeros(20195)
combfeatures.fill(0)

#cartesian((atomFeatures, combfeatures))
#print timeit(lambda:combs(combfeatures, 2), number=1)
print timeit(lambda:cartesian((combfeatures, atomFeatures)), number=1)
#data = np.array([np.zeros(20195)])
start = timer()
print data[:, idx]
end = timer()
print(end - start) 
start = timer()
np.array(np.meshgrid([2,3,1,0],[2,3,1,0])).T.reshape(-1,2)
end = timer()
print(end - start) 
#start = timer()
#list(itertools.combinations(combfeatures, 2))
#end = timer()
#print(end - start) 
iu1 = np.triu_indices(2)
a = np.array([[0,1], [0,2], [1,2]])
idx = a(np.triu_indices(n, 1)).T

In [ ]:
def test():
    count = 0
    atomFeatures = np.zeros(35)
    for i in range(0,atomFeatures.size):
        for j in range (i,atomFeatures.size):
            if(i!=j):
                if(atomFeatures[i]==0 and atomFeatures[j]==0): 
                    count+=1
def test2():
    count = 0
    atomFeatures = np.zeros(35)
    combfeatures = np.zeros(595)
    for i in range(0,atomFeatures.size):
        for j in range (i,combfeatures.size):
            if(i!=j):
                if(atomFeatures[i]==0 and combfeatures[j]==0): 
                    count+=1
    print count
def test3():
    count = 0
    atomFeatures = np.zeros(35)
    combfeatures = np.zeros(20195)
    for i in range(0,atomFeatures.size):
        for j in range (i,combfeatures.size):
            if(i!=j):
                if(atomFeatures[i]==0 and combfeatures[j]==0): 
                    count+=1
    print count
    

from timeit import timeit
timeit(lambda:test3(), number=1)

In [ ]:
from timeit import timeit
import numpy as np
#A = np.array([1,0,1])
#B = np.array([1,1,1])
A = np.zeros(35)
B = np.zeros(595)
print timeit(lambda:cysum(A,B),number=1)
print cysum(A,B)

In [ ]:
dominoes = Domino()
e = 0.03  # epsilon-greedy proportion
featureWeights = np.zeros(35)
featureComb= np.zeros(595)
state = dominoes.startGame()
features1 = Features(1,featureWeights,featureComb)
controlP1 = Control("p1",features1,"combination")
actP1 = controlP1.policyAct(state)
print actP1
controlP1.generateVector(state,actP1)
print controlP1.featuresVector
controlPP1 = Control("p1",features1,"atomic")
actPP1 = controlPP1.policyAct(state)
print actPP1
controlPP1.generateVector(state,actPP1)
print controlPP1.featuresVector